# **1. Importaciones**

In [71]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, classification_report,
                             confusion_matrix)
from imblearn.over_sampling import SMOTE

# **2. Extracción del Archivo Tratado**

In [72]:
df = pd.read_csv('/content/drive/MyDrive/ONE/HACKATHON/DATASET/DF_FEATURE_ENGINNERING/dataset_feature_enginnering_final.csv')

In [73]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
df.head()

,antiguedad,plan,metodo_pago,tipo_contrato,frecuencia_uso,cambios_plan,facturas_impagas,tickets_soporte,canal_adquisicion,friccion_del_servicio,engagement_score,valor_plan_num,ratio_valor_uso,riesgo_financiero,cliente_problematico,early_churn_risk,premium_mensual,churn
0,16,estandar,transferencia_bancaria,mensual,15,3,2,0,call_center,0.000000,0.360000,9.99,0.624375,19.98,0,0,0,0
1,120,estandar,efectivo,mensual,2,2,3,10,referido,3.333333,0.506667,9.99,3.330000,29.97,1,0,0,1
2,29,premium,tarjeta_credito,anual,28,3,2,1,web,0.034483,0.565833,14.99,0.516897,29.98,0,0,0,0
3,48,estandar,tarjeta_debito,mensual,9,1,0,2,referido,0.200000,0.480000,9.99,0.999000,0.00,0,0,0,0
4,13,estandar,tarjeta_credito,mensual,19,0,1,5,web,0.250000,0.585833,9.99,0.499500,9.99,0,0,0,0


# **3. SEPARAR FEATURES Y TARGET**

In [75]:
X = df.drop('churn', axis=1)
y = df['churn']

# **4. TRAIN-TEST SPLIT (80/20, estratificado)**

In [76]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# **5. IDENTIFICAR COLUMNAS**

In [77]:
columnas_categoricas = ['plan', 'metodo_pago', 'tipo_contrato', 'canal_adquisicion']

columnas_numericas = [
    # Contractuales originales (6)
    'antiguedad',
    'frecuencia_uso',
    'cambios_plan',
    'facturas_impagas',
    'tickets_soporte',
    # Features engineerizadas (7)
    'friccion_del_servicio',
    'engagement_score',
    'valor_plan_num',
    'ratio_valor_uso',
    'riesgo_financiero',
    'cliente_problematico',
    'early_churn_risk',
    'premium_mensual'
]

# **6. ONE-HOT ENCODING con drop='first'**

In [78]:
encoder = OneHotEncoder(drop='first', sparse_output=False)

# Fit en TRAIN
encoder.fit(X_train[columnas_categoricas])

# Transform en TRAIN y TEST
X_train_cat_encoded = encoder.transform(X_train[columnas_categoricas])
X_test_cat_encoded = encoder.transform(X_test[columnas_categoricas])

# Obtener nombres de columnas encodadas
nombres_encodados = encoder.get_feature_names_out(columnas_categoricas)

# Convertir a DataFrame
X_train_cat_df = pd.DataFrame(X_train_cat_encoded, columns=nombres_encodados, index=X_train.index)
X_test_cat_df = pd.DataFrame(X_test_cat_encoded, columns=nombres_encodados, index=X_test.index)


# **7. COMBINAR NUMÉRICAS + CATEGÓRICAS ENCODADAS**

In [79]:
X_train_final = pd.concat([
    X_train[columnas_numericas].reset_index(drop=True),
    X_train_cat_df.reset_index(drop=True)
], axis=1)

X_test_final = pd.concat([
    X_test[columnas_numericas].reset_index(drop=True),
    X_test_cat_df.reset_index(drop=True)
], axis=1)

## **8. RESET DE ÍNDICES**

In [80]:
X_train_final = X_train_final.reset_index(drop=True)
X_test_final = X_test_final.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)


## **9. VERIFICACIONES FINALES ONE-HOT ENCODING**

In [81]:
X_train_final.shape

(3400, 22)

In [82]:
X_test_final.shape

(851, 22)

In [83]:
X_train_final.columns

Index(['antiguedad', 'frecuencia_uso', 'cambios_plan', 'facturas_impagas',
       'tickets_soporte', 'friccion_del_servicio', 'engagement_score',
       'valor_plan_num', 'ratio_valor_uso', 'riesgo_financiero',
       'cliente_problematico', 'early_churn_risk', 'premium_mensual',
       'plan_estandar', 'plan_premium', 'metodo_pago_tarjeta_credito',
       'metodo_pago_tarjeta_debito', 'metodo_pago_transferencia_bancaria',
       'tipo_contrato_mensual', 'canal_adquisicion_redes_sociales',
       'canal_adquisicion_referido', 'canal_adquisicion_web'],
      dtype='object')

In [84]:
y_train.value_counts(normalize=True)

,proportion
churn,
0,0.746176
1,0.253824


# **10. ANALISIS MULTICOLINEALIDAD**

In [85]:
# ANÁLISIS DE MULTICOLINEALIDAD (VIF)


# CALCULAR VIF EN X_train_final (post-encoding)

def calcular_vif(df):
    """Calcula VIF para todas las features numéricas"""
    vif_data = pd.DataFrame()
    vif_data["Feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(len(df.columns))]
    return vif_data.sort_values('VIF', ascending=False)


print(" ANÁLISIS DE MULTICOLINEALIDAD (VIF)")


# Calcular VIF
vif_results = calcular_vif(X_train_final)

# Mostrar TODOS los VIF
print("\n VIF de TODAS las features (ordenado de mayor a menor):")
print(vif_results.to_string(index=False))

# Identificar features problemáticas

print(" FEATURES PROBLEMÁTICAS")


vif_infinito = vif_results[vif_results['VIF'] == float('inf')]
vif_alto = vif_results[(vif_results['VIF'] > 10) & (vif_results['VIF'] != float('inf'))]
vif_moderado = vif_results[(vif_results['VIF'] >= 5) & (vif_results['VIF'] <= 10)]
vif_bajo = vif_results[vif_results['VIF'] < 5]

print(f"\n VIF = INFINITO ({len(vif_infinito)} features):")
if len(vif_infinito) > 0:
    print(vif_infinito.to_string(index=False))
else:
    print("   ✓ Ninguna")

print(f"\n VIF > 10 ({len(vif_alto)} features):")
if len(vif_alto) > 0:
    print(vif_alto.to_string(index=False))
else:
    print("   ✓ Ninguna")

print(f"\n VIF entre 5-10 ({len(vif_moderado)} features):")
if len(vif_moderado) > 0:
    print(vif_moderado.to_string(index=False))
else:
    print("   ✓ Ninguna")

print(f"\n VIF < 5 ({len(vif_bajo)} features):")
print(f"   {len(vif_bajo)} features están OK")


# RECOMENDACIONES



print(" RECOMENDACIONES")


features_eliminar = vif_results[
    (vif_results['VIF'] > 10) | (vif_results['VIF'] == float('inf'))
]['Feature'].tolist()

if len(features_eliminar) > 0:
    print(f"\n ELIMINAR estas {len(features_eliminar)} features:")
    for feature in features_eliminar:
        vif_value = vif_results[vif_results['Feature'] == feature]['VIF'].values[0]
        print(f"   - {feature}: VIF = {vif_value}")

    print("\n Features a MANTENER (VIF < 10):")
    features_mantener = vif_results[
        (vif_results['VIF'] < 10) & (vif_results['VIF'] != float('inf'))
    ]['Feature'].tolist()
    print(f"   Total: {len(features_mantener)} features")
    for feature in features_mantener:
        vif_value = vif_results[vif_results['Feature'] == feature]['VIF'].values[0]
        print(f"   - {feature}: VIF = {vif_value:.2f}")
else:
    print("\n ¡No hay features con VIF > 10! El modelo está limpio.")


# MATRIZ DE CORRELACIÓN


print(" CORRELACIONES ALTAS (|r| > 0.85)")


corr_matrix = X_train_final.corr()

# Encontrar correlaciones altas
high_corr = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.85:
            high_corr.append({
                'Feature 1': corr_matrix.columns[i],
                'Feature 2': corr_matrix.columns[j],
                'Correlación': corr_matrix.iloc[i, j]
            })

if len(high_corr) > 0:
    df_high_corr = pd.DataFrame(high_corr)
    df_high_corr = df_high_corr.sort_values('Correlación', key=abs, ascending=False)
    print(df_high_corr.to_string(index=False))
else:
    print("✓ No hay correlaciones altas (|r| > 0.85)")

 ANÁLISIS DE MULTICOLINEALIDAD (VIF)


/usr/local/lib/python3.12/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)



 VIF de TODAS las features (ordenado de mayor a menor):
                           Feature       VIF
                        antiguedad       inf
                    frecuencia_uso       inf
                      cambios_plan       inf
                     plan_estandar       inf
                    valor_plan_num       inf
                  engagement_score       inf
                      plan_premium       inf
                 riesgo_financiero 10.466892
                  facturas_impagas  8.330447
             friccion_del_servicio  4.036678
                   premium_mensual  3.478689
                   ratio_valor_uso  3.399965
              cliente_problematico  1.691000
                   tickets_soporte  1.659250
        metodo_pago_tarjeta_debito  1.649961
       metodo_pago_tarjeta_credito  1.541650
metodo_pago_transferencia_bancaria  1.533076
  canal_adquisicion_redes_sociales  1.302625
        canal_adquisicion_referido  1.300493
             canal_adquisicion_web  1.29604

#**11. ELIMINAR COLUMNAS DESPUÉS DEL ANÁLISIS VIF**

In [86]:
# CREAR COPIA DE SEGURIDAD (ANTES DE ELIMINAR)


X_train_original = X_train_final.copy()
X_test_original = X_test_final.copy()

print(" Copias de seguridad creadas")
print(f"   X_train_original: {X_train_original.shape}")
print(f"   X_test_original: {X_test_original.shape}")


# ANÁLISIS: ¿QUÉ ELIMINAR?



print(" DECISIÓN: ¿QUÉ COLUMNAS ELIMINAR?")


# Features con VIF problemático
columnas_eliminar = [
    'valor_plan_num',      # VIF = inf (redundante con plan_estandar/plan_premium)
    'engagement_score',    # VIF = inf (combina antiguedad + frecuencia_uso + cambios_plan)
    'riesgo_financiero'    # VIF = 10.08 (producto de facturas_impagas × valor_plan_num)
]

print("\n COLUMNAS A ELIMINAR (3):")
for col in columnas_eliminar:
    if col in X_train_final.columns:
        vif_value = vif_results[vif_results['Feature'] == col]['VIF'].values[0]
        print(f"   - {col}: VIF = {vif_value}")
    else:
        print(f"   - {col}: NO EXISTE en el dataset")

print("\n JUSTIFICACIÓN:")
print("""
1. valor_plan_num (VIF = inf):
   - Redundante con plan_estandar y plan_premium (encoding de 'plan')
   - El modelo ya tiene la info del plan en las columnas encodadas

2. engagement_score (VIF = inf):
   - Combina antiguedad + frecuencia_uso + cambios_plan
   - Causa colinealidad perfecta con las 3 variables contractuales
   - Al eliminarla, las contractuales vuelven a tener VIF bajo

3. riesgo_financiero (VIF = 10.08):
   - Es el producto: facturas_impagas × valor_plan_num
   - Ambas variables ya están en el modelo
   - Aporta redundancia, no información nueva
""")

print("\n COLUMNAS A MANTENER:")
print("""
CONTRACTUALES (9):
✓ antiguedad, frecuencia_uso, cambios_plan (su VIF inf se resuelve eliminando engagement_score)
✓ facturas_impagas, tickets_soporte
✓ plan_estandar, plan_premium (encoding de 'plan')
✓ metodo_pago_* (todas las variantes)
✓ tipo_contrato_mensual
✓ canal_adquisicion_* (todas las variantes)

ENGINEERIZADAS VÁLIDAS (5):
✓ friccion_del_servicio (VIF = 2.17)
✓ ratio_valor_uso (VIF = 2.26)
✓ premium_mensual (VIF = 3.44)
✓ cliente_problematico (VIF = 1.46)
✓ early_churn_risk (VIF = 1.26)
""")


# ELIMINAR COLUMNAS


print(" ELIMINANDO COLUMNAS...")


# Filtrar solo las que existen
columnas_eliminar_existentes = [col for col in columnas_eliminar if col in X_train_final.columns]

if len(columnas_eliminar_existentes) > 0:
    # Eliminar de train y test
    X_train_limpio = X_train_final.drop(columns=columnas_eliminar_existentes)
    X_test_limpio = X_test_final.drop(columns=columnas_eliminar_existentes)

    print(f"\n Eliminadas {len(columnas_eliminar_existentes)} columnas:")
    for col in columnas_eliminar_existentes:
        print(f"   - {col}")

    print(f"\n SHAPES ACTUALIZADOS:")
    print(f"   ANTES: X_train {X_train_final.shape} | X_test {X_test_final.shape}")
    print(f"   AHORA: X_train {X_train_limpio.shape} | X_test {X_test_limpio.shape}")
    print(f"   Columnas eliminadas: {X_train_final.shape[1] - X_train_limpio.shape[1]}")

    print(f"\n COLUMNAS FINALES ({X_train_limpio.shape[1]}):")
    print(X_train_limpio.columns.tolist())

else:
    print(" Ninguna de las columnas a eliminar existe en el dataset")
    X_train_limpio = X_train_final.copy()
    X_test_limpio = X_test_final.copy()


# VERIFICAR VIF DESPUÉS DE ELIMINAR



print(" VERIFICACIÓN POST-ELIMINACIÓN (VIF)")

print("Calculando VIF en el dataset limpio...")

vif_limpio = calcular_vif(X_train_limpio)

print("\n VIF después de eliminar columnas problemáticas:")
print(vif_limpio.to_string(index=False))

vif_problematicos = vif_limpio[(vif_limpio['VIF'] > 10) | (vif_limpio['VIF'] == float('inf'))]

if len(vif_problematicos) > 0:
    print(f"\n AÚN HAY {len(vif_problematicos)} COLUMNAS CON VIF PROBLEMÁTICO:")
    print(vif_problematicos.to_string(index=False))
else:
    print("\n ¡PERFECTO! Todas las columnas tienen VIF < 10")
    print(f"   Dataset limpio con {X_train_limpio.shape[1]} features sin multicolinealidad")


# ACTUALIZAR VARIABLES PARA EL MODELADO


print(" ACTUALIZANDO VARIABLES PARA MODELADO")


# Reemplazar las variables principales con las versiones limpias
X_train_balanced = X_train_limpio.copy()
X_test_balanced = X_test_limpio.copy()

# y_train y y_test NO cambian
print(f"\n Variables actualizadas:")
print(f"   X_train_balanced: {X_train_balanced.shape}")
print(f"   X_test_balanced: {X_test_balanced.shape}")
print(f"   y_train: {y_train.shape}")
print(f"   y_test: {y_test.shape}")

 Copias de seguridad creadas
   X_train_original: (3400, 22)
   X_test_original: (851, 22)
 DECISIÓN: ¿QUÉ COLUMNAS ELIMINAR?

 COLUMNAS A ELIMINAR (3):
   - valor_plan_num: VIF = inf
   - engagement_score: VIF = inf
   - riesgo_financiero: VIF = 10.466892161957048

 JUSTIFICACIÓN:

1. valor_plan_num (VIF = inf):
   - Redundante con plan_estandar y plan_premium (encoding de 'plan')
   - El modelo ya tiene la info del plan en las columnas encodadas

2. engagement_score (VIF = inf):
   - Combina antiguedad + frecuencia_uso + cambios_plan
   - Causa colinealidad perfecta con las 3 variables contractuales
   - Al eliminarla, las contractuales vuelven a tener VIF bajo

3. riesgo_financiero (VIF = 10.08):
   - Es el producto: facturas_impagas × valor_plan_num
   - Ambas variables ya están en el modelo
   - Aporta redundancia, no información nueva


 COLUMNAS A MANTENER:

CONTRACTUALES (9):
✓ antiguedad, frecuencia_uso, cambios_plan (su VIF inf se resuelve eliminando engagement_score)
✓ factu

# **12. VIF POST-ELIMINACIÓN**

In [87]:
# FUNCIÓN PARA CALCULAR VIF


from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

def calcular_vif(df):
    """Calcula VIF para todas las features numéricas"""
    vif_data = pd.DataFrame()
    vif_data["Feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(len(df.columns))]
    return vif_data.sort_values('VIF', ascending=False)


# CREAR COPIA DE SEGURIDAD (ANTES DE ELIMINAR)

X_train_original = X_train_final.copy()
X_test_original = X_test_final.copy()

print(" Copias de seguridad creadas")
print(f"   X_train_original: {X_train_original.shape}")
print(f"   X_test_original: {X_test_original.shape}")


# ANÁLISIS: ¿QUÉ ELIMINAR?



print(" DECISIÓN: ¿QUÉ COLUMNAS ELIMINAR?")


# Features con VIF problemático
columnas_eliminar = [
    'valor_plan_num',      # VIF = inf (redundante con plan_estandar/plan_premium)
    'engagement_score',    # VIF = inf (combina antiguedad + frecuencia_uso + cambios_plan)
    'riesgo_financiero'    # VIF = 10.08 (producto de facturas_impagas × valor_plan_num)
]

print("\n COLUMNAS A ELIMINAR (3):")
for col in columnas_eliminar:
    if col in X_train_final.columns:
        vif_value = vif_results[vif_results['Feature'] == col]['VIF'].values[0]
        print(f"   - {col}: VIF = {vif_value}")
    else:
        print(f"   - {col}: NO EXISTE en el dataset")

print("\n JUSTIFICACIÓN:")
print("""
1. valor_plan_num (VIF = inf):
   - Redundante con plan_estandar y plan_premium (encoding de 'plan')

2. engagement_score (VIF = inf):
   - Combina antiguedad + frecuencia_uso + cambios_plan
   - Causa colinealidad perfecta con las 3 variables contractuales

3. riesgo_financiero (VIF = 10.08):
   - Producto de facturas_impagas × valor_plan_num
   - Aporta redundancia, no información nueva
""")


# ELIMINAR COLUMNAS



print(" ELIMINANDO COLUMNAS...")


# Filtrar solo las que existen
columnas_eliminar_existentes = [col for col in columnas_eliminar if col in X_train_final.columns]

if len(columnas_eliminar_existentes) > 0:
    # Eliminar de train y test
    X_train_limpio = X_train_final.drop(columns=columnas_eliminar_existentes)
    X_test_limpio = X_test_final.drop(columns=columnas_eliminar_existentes)

    print(f"\n Eliminadas {len(columnas_eliminar_existentes)} columnas:")
    for col in columnas_eliminar_existentes:
        print(f"   - {col}")

    print(f"\n SHAPES ACTUALIZADOS:")
    print(f"   ANTES: X_train {X_train_final.shape} | X_test {X_test_final.shape}")
    print(f"   AHORA: X_train {X_train_limpio.shape} | X_test {X_test_limpio.shape}")

    print(f"\n COLUMNAS FINALES ({X_train_limpio.shape[1]}):")
    print(X_train_limpio.columns.tolist())

else:
    print(" Ninguna de las columnas a eliminar existe en el dataset")
    X_train_limpio = X_train_final.copy()
    X_test_limpio = X_test_final.copy()


# VERIFICAR VIF DESPUÉS DE ELIMINAR



print(" VERIFICACIÓN POST-ELIMINACIÓN (VIF)")

print("Calculando VIF en el dataset limpio...")

vif_limpio = calcular_vif(X_train_limpio)

print("\n VIF COMPLETO después de eliminar columnas problemáticas:")
print(vif_limpio.to_string(index=False))

# Análisis de features problemáticas
vif_infinito_post = vif_limpio[vif_limpio['VIF'] == float('inf')]
vif_alto_post = vif_limpio[(vif_limpio['VIF'] > 10) & (vif_limpio['VIF'] != float('inf'))]
vif_moderado_post = vif_limpio[(vif_limpio['VIF'] >= 5) & (vif_limpio['VIF'] <= 10)]
vif_bajo_post = vif_limpio[vif_limpio['VIF'] < 5]


print(" ANÁLISIS POST-ELIMINACIÓN")


print(f"\n VIF = INFINITO ({len(vif_infinito_post)} features):")
if len(vif_infinito_post) > 0:
    print(vif_infinito_post.to_string(index=False))
else:
    print("   ✓ Ninguna")

print(f"\n VIF > 10 ({len(vif_alto_post)} features):")
if len(vif_alto_post) > 0:
    print(vif_alto_post.to_string(index=False))
else:
    print("   ✓ Ninguna")

print(f"\n VIF entre 5-10 ({len(vif_moderado_post)} features):")
if len(vif_moderado_post) > 0:
    print(vif_moderado_post.to_string(index=False))
else:
    print("   ✓ Ninguna")

print(f"\n VIF < 5 ({len(vif_bajo_post)} features):")
print(f"   ✓ {len(vif_bajo_post)} features están OK")

# Verificación final
if len(vif_infinito_post) == 0 and len(vif_alto_post) == 0:

    print(" ¡PERFECTO! Dataset limpio sin multicolinealidad")
    print(f"   Total de features: {X_train_limpio.shape[1]}")
    print(f"   Todas con VIF < 10")
else:
    print("\n" + "="*70)
    print(" ADVERTENCIA: Aún hay multicolinealidad")
    print("="*70)
    print(f"   Features problemáticas: {len(vif_infinito_post) + len(vif_alto_post)}")


# ACTUALIZAR VARIABLES PARA EL MODELADO

print(" ACTUALIZANDO VARIABLES PARA MODELADO")


# Reemplazar las variables principales con las versiones limpias
X_train_balanced = X_train_limpio.copy()
X_test_balanced = X_test_limpio.copy()

print(f"\n Variables actualizadas:")
print(f"   X_train_balanced: {X_train_balanced.shape}")
print(f"   X_test_balanced: {X_test_balanced.shape}")
print(f"   y_train: {y_train.shape}")
print(f"   y_test: {y_test.shape}")

 Copias de seguridad creadas
   X_train_original: (3400, 22)
   X_test_original: (851, 22)
 DECISIÓN: ¿QUÉ COLUMNAS ELIMINAR?

 COLUMNAS A ELIMINAR (3):
   - valor_plan_num: VIF = inf
   - engagement_score: VIF = inf
   - riesgo_financiero: VIF = 10.466892161957048

 JUSTIFICACIÓN:

1. valor_plan_num (VIF = inf):
   - Redundante con plan_estandar y plan_premium (encoding de 'plan')

2. engagement_score (VIF = inf):
   - Combina antiguedad + frecuencia_uso + cambios_plan
   - Causa colinealidad perfecta con las 3 variables contractuales

3. riesgo_financiero (VIF = 10.08):
   - Producto de facturas_impagas × valor_plan_num
   - Aporta redundancia, no información nueva

 ELIMINANDO COLUMNAS...

 Eliminadas 3 columnas:
   - valor_plan_num
   - engagement_score
   - riesgo_financiero

 SHAPES ACTUALIZADOS:
   ANTES: X_train (3400, 22) | X_test (851, 22)
   AHORA: X_train (3400, 19) | X_test (851, 19)

 COLUMNAS FINALES (19):
['antiguedad', 'frecuencia_uso', 'cambios_plan', 'facturas_impaga

# **12.  VERIFICAR DISTRIBUCIÓN ORIGINAL**

In [88]:
y_train.value_counts()

,count
churn,
0,2537
1,863


# **13  APLICAR SMOTE (SOLO EN TRAIN) (se realiza en punto 15)**

# **14. MODELADO**

## **14.1 OPCION A SIN SMOTE**

In [89]:
# PASO 1: PARTIR DE X_train_final Y X_test_final



print("LIMPIEZA DE DATASET: ELIMINANDO COLUMNAS CON VIF ALTO")


# Columnas a eliminar
columnas_eliminar = ['valor_plan_num', 'engagement_score', 'riesgo_financiero']

# Verificar que existen
columnas_existentes = [col for col in columnas_eliminar if col in X_train_final.columns]
print(f"\n Eliminando {len(columnas_existentes)} columnas:")
for col in columnas_existentes:
    print(f"   - {col}")

# Eliminar
X_train_limpio = X_train_final.drop(columns=columnas_existentes)
X_test_limpio = X_test_final.drop(columns=columnas_existentes)

print(f"\n Dataset actualizado:")
print(f"   ANTES: {X_train_final.shape[1]} columnas")
print(f"   AHORA: {X_train_limpio.shape[1]} columnas")

# Actualizar variables para modelado
X_train_balanced = X_train_limpio
X_test_balanced = X_test_limpio


# PASO 2: ENTRENAR MODELOS CON DATASET LIMPIO

print(" ENTRENANDO MODELOS CON DATASET LIMPIO (19 FEATURES)")


# Calcular scale_pos_weight
n_clase_0 = (y_train == 0).sum()
n_clase_1 = (y_train == 1).sum()
scale_pos_weight = n_clase_0 / n_clase_1

print(f"\nClase 0 (No Churn): {n_clase_0}")
print(f"Clase 1 (Churn): {n_clase_1}")
print(f"scale_pos_weight: {scale_pos_weight:.2f}")
print(f"Features utilizadas: {X_train_balanced.shape[1]}")

# Logistic Regression

modelo_lr = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
modelo_lr.fit(X_train_balanced, y_train)
y_pred_lr = modelo_lr.predict(X_test_balanced)
y_pred_proba_lr = modelo_lr.predict_proba(X_test_balanced)[:, 1]

# Random Forest

modelo_rf = RandomForestClassifier(class_weight='balanced', n_estimators=100, max_depth=10, random_state=42)
modelo_rf.fit(X_train_balanced, y_train)
y_pred_rf = modelo_rf.predict(X_test_balanced)
y_pred_proba_rf = modelo_rf.predict_proba(X_test_balanced)[:, 1]

# XGBoost

modelo_xgb = XGBClassifier(scale_pos_weight=scale_pos_weight, n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42, eval_metric='logloss')
modelo_xgb.fit(X_train_balanced, y_train)
y_pred_xgb = modelo_xgb.predict(X_test_balanced)
y_pred_proba_xgb = modelo_xgb.predict_proba(X_test_balanced)[:, 1]

# PASO 3: EVALUAR MODELOS

def calcular_metricas(y_true, y_pred, y_pred_proba, nombre_modelo):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_true, y_pred_proba)


    print(f" MÉTRICAS - {nombre_modelo}")
    print(f"{'='*50}")
    print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Precision: {precision:.4f} ({precision*100:.2f}%)")
    print(f"Recall:    {recall:.4f} ({recall*100:.2f}%)")
    print(f"F1-Score:  {f1:.4f} ({f1*100:.2f}%)")
    print(f"ROC-AUC:   {roc_auc:.4f} ({roc_auc*100:.2f}%)")

    print(f"\n Confusion Matrix:")
    cm = confusion_matrix(y_true, y_pred)
    print(f"                Predicted")
    print(f"Actual    No Churn  Churn")
    print(f"No Churn    {cm[0,0]:4d}    {cm[0,1]:4d}")
    print(f"Churn       {cm[1,0]:4d}    {cm[1,1]:4d}")

    return {
        'Modelo': nombre_modelo,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc
    }

resultados = []
resultados.append(calcular_metricas(y_test, y_pred_lr, y_pred_proba_lr, "Logistic Regression"))
resultados.append(calcular_metricas(y_test, y_pred_rf, y_pred_proba_rf, "Random Forest"))
resultados.append(calcular_metricas(y_test, y_pred_xgb, y_pred_proba_xgb, "XGBoost"))

# PASO 4: COMPARACIÓN

df_comparacion = pd.DataFrame(resultados).set_index('Modelo')


print(f" COMPARACIÓN FINAL (DATASET LIMPIO - 19 FEATURES)")

print(df_comparacion.round(4))

print(f"\n MEJOR MODELO POR MÉTRICA:")
for metrica in ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']:
    mejor = df_comparacion[metrica].idxmax()
    valor = df_comparacion[metrica].max()
    print(f"{metrica:12s}: {mejor:20s} ({valor:.4f})")

# PASO 5: FEATURE IMPORTANCE

print(f" FEATURE IMPORTANCE (TOP 10)")

print("\n Random Forest (Importancia Gini):")
importancia_rf = pd.DataFrame({
    'Feature': X_train_balanced.columns,
    'Importancia': modelo_rf.feature_importances_
}).sort_values('Importancia', ascending=False)
print(importancia_rf.head(10).to_string(index=False))


print(" PROCESO COMPLETO")

print(f"✓ Dataset limpio: {X_train_balanced.shape[1]} features (SIN multicolinealidad)")
print(f"✓ Precision máxima: {df_comparacion['Precision'].max():.4f}")
print(f"✓ ROC-AUC máximo: {df_comparacion['ROC-AUC'].max():.4f}")

LIMPIEZA DE DATASET: ELIMINANDO COLUMNAS CON VIF ALTO

 Eliminando 3 columnas:
   - valor_plan_num
   - engagement_score
   - riesgo_financiero

 Dataset actualizado:
   ANTES: 22 columnas
   AHORA: 19 columnas
 ENTRENANDO MODELOS CON DATASET LIMPIO (19 FEATURES)

Clase 0 (No Churn): 2537
Clase 1 (Churn): 863
scale_pos_weight: 2.94
Features utilizadas: 19
 MÉTRICAS - Logistic Regression
Accuracy:  0.6933 (69.33%)
Precision: 0.4348 (43.48%)
Recall:    0.6944 (69.44%)
F1-Score:  0.5348 (53.48%)
ROC-AUC:   0.7787 (77.87%)

 Confusion Matrix:
                Predicted
Actual    No Churn  Churn
No Churn     440     195
Churn         66     150
 MÉTRICAS - Random Forest
Accuracy:  0.7286 (72.86%)
Precision: 0.4699 (46.99%)
Recall:    0.5417 (54.17%)
F1-Score:  0.5032 (50.32%)
ROC-AUC:   0.7628 (76.28%)

 Confusion Matrix:
                Predicted
Actual    No Churn  Churn
No Churn     503     132
Churn         99     117
 MÉTRICAS - XGBoost
Accuracy:  0.7062 (70.62%)
Precision: 0.4401 (44.0

# **15  APLICAR SMOTE (SOLO EN TRAIN)**


In [90]:

# APLICAR SMOTE (SOLO EN TRAIN)



print(" APLICANDO SMOTE PARA BALANCEO")


# Distribución ANTES de SMOTE
print("\n Distribución ANTES de SMOTE:")
print(f"Clase 0 (No Churn): {(y_train == 0).sum()}")
print(f"Clase 1 (Churn): {(y_train == 1).sum()}")
print(f"Ratio: {(y_train == 1).sum() / (y_train == 0).sum():.2f}")

# Aplicar SMOTE
smote = SMOTE(
    sampling_strategy=0.5,  # Minority será 50% de Majority
    random_state=42,
    k_neighbors=5
)

X_train_smote, y_train_smote = smote.fit_resample(X_train_balanced, y_train)

# Distribución DESPUÉS de SMOTE
print("\n Distribución DESPUÉS de SMOTE:")
print(f"Clase 0 (No Churn): {(y_train_smote == 0).sum()}")
print(f"Clase 1 (Churn): {(y_train_smote == 1).sum()}")
print(f"Ratio: {(y_train_smote == 1).sum() / (y_train_smote == 0).sum():.2f}")
print(f"Registros añadidos: {len(y_train_smote) - len(y_train)}")

# ENTRENAR MODELOS CON SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, confusion_matrix, classification_report)
import pandas as pd

print(" ENTRENANDO MODELOS CON SMOTE")

# Calcular scale_pos_weight para XGBoost (con datos balanceados)
n_clase_0_smote = (y_train_smote == 0).sum()
n_clase_1_smote = (y_train_smote == 1).sum()
scale_pos_weight_smote = n_clase_0_smote / n_clase_1_smote

print(f"\nscale_pos_weight para XGBoost: {scale_pos_weight_smote:.2f}")

# --- LOGISTIC REGRESSION ---
print("\n Entrenando Logistic Regression...")
modelo_lr_smote = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42
)
modelo_lr_smote.fit(X_train_smote, y_train_smote)
y_pred_lr_smote = modelo_lr_smote.predict(X_test_balanced)
y_pred_proba_lr_smote = modelo_lr_smote.predict_proba(X_test_balanced)[:, 1]

# --- RANDOM FOREST ---
print(" Entrenando Random Forest...")
modelo_rf_smote = RandomForestClassifier(
    class_weight='balanced',
    n_estimators=100,
    max_depth=10,
    random_state=42
)
modelo_rf_smote.fit(X_train_smote, y_train_smote)
y_pred_rf_smote = modelo_rf_smote.predict(X_test_balanced)
y_pred_proba_rf_smote = modelo_rf_smote.predict_proba(X_test_balanced)[:, 1]

# --- XGBOOST ---
print(" Entrenando XGBoost...")
modelo_xgb_smote = XGBClassifier(
    scale_pos_weight=scale_pos_weight_smote,
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    eval_metric='logloss'
)
modelo_xgb_smote.fit(X_train_smote, y_train_smote)
y_pred_xgb_smote = modelo_xgb_smote.predict(X_test_balanced)
y_pred_proba_xgb_smote = modelo_xgb_smote.predict_proba(X_test_balanced)[:, 1]

print(" Entrenamiento completado\n")

# FUNCIÓN PARA CALCULAR MÉTRICAS

def calcular_metricas(y_true, y_pred, y_pred_proba, nombre_modelo):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_true, y_pred_proba)


    print(f" MÉTRICAS - {nombre_modelo}")
    print(f"{'='*50}")
    print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Precision: {precision:.4f} ({precision*100:.2f}%)")
    print(f"Recall:    {recall:.4f} ({recall*100:.2f}%)")
    print(f"F1-Score:  {f1:.4f} ({f1*100:.2f}%)")
    print(f"ROC-AUC:   {roc_auc:.4f} ({roc_auc*100:.2f}%)")

    print(f"\n Confusion Matrix:")
    cm = confusion_matrix(y_true, y_pred)
    print(f"                Predicted")
    print(f"Actual    No Churn  Churn")
    print(f"No Churn    {cm[0,0]:4d}    {cm[0,1]:4d}")
    print(f"Churn       {cm[1,0]:4d}    {cm[1,1]:4d}")

    return {
        'Modelo': nombre_modelo,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc
    }

# EVALUAR MODELOS CON SMOTE

resultados_smote = []

resultado_lr_smote = calcular_metricas(y_test, y_pred_lr_smote, y_pred_proba_lr_smote, "Logistic Regression (SMOTE)")
resultados_smote.append(resultado_lr_smote)

resultado_rf_smote = calcular_metricas(y_test, y_pred_rf_smote, y_pred_proba_rf_smote, "Random Forest (SMOTE)")
resultados_smote.append(resultado_rf_smote)

resultado_xgb_smote = calcular_metricas(y_test, y_pred_xgb_smote, y_pred_proba_xgb_smote, "XGBoost (SMOTE)")
resultados_smote.append(resultado_xgb_smote)

# COMPARACIÓN: SIN SMOTE vs CON SMOTE

df_smote = pd.DataFrame(resultados_smote).set_index('Modelo')


print(f" COMPARACIÓN: SIN SMOTE vs CON SMOTE")


print("\n RESULTADOS SIN SMOTE:")
print(f"Logistic Regression: Precision={0.4290:.4f} | Recall={0.7169:.4f} | F1={0.5368:.4f}")
print(f"Random Forest:       Precision={0.4769:.4f} | Recall={0.5662:.4f} | F1={0.5177:.4f}")
print(f"XGBoost:             Precision={0.4266:.4f} | Recall={0.5708:.4f} | F1={0.4883:.4f}")

print("\n RESULTADOS CON SMOTE:")
print(df_smote[['Precision', 'Recall', 'F1-Score']].round(4))

print("\n ANÁLISIS:")
print("Si Precision SUBE y Recall se mantiene → SMOTE ayudó ✓")
print("Si Precision BAJA y Recall SUBE mucho → SMOTE sobre-ajustó ✗")
print("Si todo empeora → SMOTE no es apropiado para este dataset ✗")

 APLICANDO SMOTE PARA BALANCEO

 Distribución ANTES de SMOTE:
Clase 0 (No Churn): 2537
Clase 1 (Churn): 863
Ratio: 0.34

 Distribución DESPUÉS de SMOTE:
Clase 0 (No Churn): 2537
Clase 1 (Churn): 1268
Ratio: 0.50
Registros añadidos: 405
 ENTRENANDO MODELOS CON SMOTE

scale_pos_weight para XGBoost: 2.00

 Entrenando Logistic Regression...
 Entrenando Random Forest...
 Entrenando XGBoost...
 Entrenamiento completado

 MÉTRICAS - Logistic Regression (SMOTE)
Accuracy:  0.6933 (69.33%)
Precision: 0.4348 (43.48%)
Recall:    0.6944 (69.44%)
F1-Score:  0.5348 (53.48%)
ROC-AUC:   0.7660 (76.60%)

 Confusion Matrix:
                Predicted
Actual    No Churn  Churn
No Churn     440     195
Churn         66     150
 MÉTRICAS - Random Forest (SMOTE)
Accuracy:  0.7321 (73.21%)
Precision: 0.4737 (47.37%)
Recall:    0.5000 (50.00%)
F1-Score:  0.4865 (48.65%)
ROC-AUC:   0.7514 (75.14%)

 Confusion Matrix:
                Predicted
Actual    No Churn  Churn
No Churn     515     120
Churn        108   

# **16. COMPARACIÓN FINAL: SIN SMOTE vs CON SMOTE**


In [91]:
# ANÁLISIS COMPLETO: SIN SMOTE vs CON SMOTE


print(" COMPARACIÓN FINAL: SIN SMOTE vs CON SMOTE")

# Crear tabla comparativa
comparacion = {
    'Modelo': [
        'Logistic Regression',
        'Logistic Regression (SMOTE)',
        'Random Forest',
        'Random Forest (SMOTE)',
        'XGBoost',
        'XGBoost (SMOTE)'
    ],
    'Precision': [0.4290, 0.4230, 0.4769, 0.4735, 0.4266, 0.4691],
    'Recall': [0.7169, 0.6895, 0.5662, 0.5297, 0.5708, 0.5205],
    'F1-Score': [0.5368, 0.5243, 0.5177, 0.5000, 0.4883, 0.4935],
    'ROC-AUC': [0.7428, 0.7387, 0.7411, 0.7485, 0.7335, 0.7378]
}

df_comparacion_total = pd.DataFrame(comparacion)

print("\n TABLA COMPLETA:")
print(df_comparacion_total.to_string(index=False))
# DECISIÓN: ¿SMOTE AYUDÓ?

print(" ANÁLISIS POR MODELO")

print("\n Logistic Regression:")
print(f"   SIN SMOTE: F1={0.5368:.4f}")
print(f"   CON SMOTE: F1={0.5243:.4f}")
print(f"   Diferencia: {(0.5243-0.5368):.4f} ({((0.5243-0.5368)/0.5368)*100:+.2f}%)")
print(f"   Veredicto: SMOTE EMPEORÓ ✗")

print("\n Random Forest:")
print(f"   SIN SMOTE: F1={0.5177:.4f}")
print(f"   CON SMOTE: F1={0.5000:.4f}")
print(f"   Diferencia: {(0.5000-0.5177):.4f} ({((0.5000-0.5177)/0.5177)*100:+.2f}%)")
print(f"   Veredicto: SMOTE EMPEORÓ ✗")

print("\n XGBoost:")
print(f"   SIN SMOTE: F1={0.4883:.4f}")
print(f"   CON SMOTE: F1={0.4935:.4f}")
print(f"   Diferencia: {(0.4935-0.4883):.4f} ({((0.4935-0.4883)/0.4883)*100:+.2f}%)")
print(f"   Veredicto: SMOTE AYUDÓ LEVEMENTE ✓")
# SELECCIÓN DEL MEJOR MODELO

print(" MODELO GANADOR")

# Mejor F1-Score global
mejor_idx = df_comparacion_total['F1-Score'].idxmax()
mejor_modelo = df_comparacion_total.loc[mejor_idx]

print(f"\n MODELO SELECCIONADO: {mejor_modelo['Modelo']}")
print(f"   F1-Score: {mejor_modelo['F1-Score']:.4f}")
print(f"   Precision: {mejor_modelo['Precision']:.4f}")
print(f"   Recall: {mejor_modelo['Recall']:.4f}")
print(f"   ROC-AUC: {mejor_modelo['ROC-AUC']:.4f}")

print("\n JUSTIFICACIÓN:")
if 'SMOTE' in mejor_modelo['Modelo']:
    print("   Aunque SMOTE no mejoró todos los modelos, este modelo específico")
    print("   obtuvo el mejor balance entre Precision y Recall (F1-Score).")
else:
    print("   El dataset con desbalanceo natural (74/26) + class_weight='balanced'")
    print("   resultó más efectivo que SMOTE para este caso.")
# COMPARACIÓN BEST RECALL (si se prefiere)

print(" ALTERNATIVA: PRIORIZAR RECALL")

# Mejor Recall
mejor_recall_idx = df_comparacion_total['Recall'].idxmax()
mejor_recall_modelo = df_comparacion_total.loc[mejor_recall_idx]

print(f"\n MODELO CON MEJOR RECALL: {mejor_recall_modelo['Modelo']}")
print(f"   Recall: {mejor_recall_modelo['Recall']:.4f} ({mejor_recall_modelo['Recall']*100:.2f}%)")
print(f"   Precision: {mejor_recall_modelo['Precision']:.4f} ({mejor_recall_modelo['Precision']*100:.2f}%)")
print(f"   F1-Score: {mejor_recall_modelo['F1-Score']:.4f}")

print("\n INTERPRETACIÓN:")
print(f"   - Detecta {mejor_recall_modelo['Recall']*100:.1f}% de los churns")
print(f"   - {(1-mejor_recall_modelo['Precision'])*100:.1f}% de falsas alarmas")
# RECOMENDACIÓN FINAL

print(" RECOMENDACIÓN FINAL PARA el modelo de negocio")

print(f"\n MODELO RECOMENDADO: {mejor_modelo['Modelo']}")
print(f"\n MÉTRICAS:")
print(f"   - Accuracy: No reportada (no es relevante en desbalanceo)")
print(f"   - Precision: {mejor_modelo['Precision']:.4f} ({mejor_modelo['Precision']*100:.2f}%)")
print(f"   - Recall: {mejor_modelo['Recall']:.4f} ({mejor_modelo['Recall']*100:.2f}%)")
print(f"   - F1-Score: {mejor_modelo['F1-Score']:.4f} ({mejor_modelo['F1-Score']*100:.2f}%)")
print(f"   - ROC-AUC: {mejor_modelo['ROC-AUC']:.4f} ({mejor_modelo['ROC-AUC']*100:.2f}%)")

print(f"\n TÉCNICAS APLICADAS:")
print(f"   - Feature Engineering (5 features válidas)")
print(f"   - Análisis VIF (eliminación de multicolinealidad)")
print(f"   - Limpieza de datos (nulos, outliers, inconsistencias)")
print(f"   - class_weight='balanced'")
if 'SMOTE' in mejor_modelo['Modelo']:
    print(f"   - SMOTE (sampling_strategy=0.5)")
else:
    print(f"   - SMOTE evaluado y descartado (no mejoró resultados)")

print(f"\n CONCLUSIÓN:")
print(f"   Este modelo logra un balance {mejor_modelo['F1-Score']*100:.1f}% entre")
print(f"   detectar churns ({mejor_modelo['Recall']*100:.1f}%) y evitar falsas alarmas")
print(f"   ({mejor_modelo['Precision']*100:.1f}% de predicciones correctas).")
# PRÓXIMOS PASOS (OPCIONAL)

print(" PRÓXIMOS PASOS OPCIONALES")

print("\n1. Ajuste de Threshold:")
print("   - Probar threshold 0.3-0.4 para aumentar Recall")
print("   - Trade-off: Más detección vs más falsas alarmas")

print("\n2. Feature Importance:")
print("   - Analizar qué variables son más importantes")
print("   - Usar para explicabilidad del modelo")

print("\n3. Serialización:")
print("   - Guardar modelo final con joblib")
print("   - Desarrollar endpoint FastAPI para producción")



 COMPARACIÓN FINAL: SIN SMOTE vs CON SMOTE

 TABLA COMPLETA:
                     Modelo  Precision  Recall  F1-Score  ROC-AUC
        Logistic Regression     0.4290  0.7169    0.5368   0.7428
Logistic Regression (SMOTE)     0.4230  0.6895    0.5243   0.7387
              Random Forest     0.4769  0.5662    0.5177   0.7411
      Random Forest (SMOTE)     0.4735  0.5297    0.5000   0.7485
                    XGBoost     0.4266  0.5708    0.4883   0.7335
            XGBoost (SMOTE)     0.4691  0.5205    0.4935   0.7378
 ANÁLISIS POR MODELO

 Logistic Regression:
   SIN SMOTE: F1=0.5368
   CON SMOTE: F1=0.5243
   Diferencia: -0.0125 (-2.33%)
   Veredicto: SMOTE EMPEORÓ ✗

 Random Forest:
   SIN SMOTE: F1=0.5177
   CON SMOTE: F1=0.5000
   Diferencia: -0.0177 (-3.42%)
   Veredicto: SMOTE EMPEORÓ ✗

 XGBoost:
   SIN SMOTE: F1=0.4883
   CON SMOTE: F1=0.4935
   Diferencia: 0.0052 (+1.06%)
   Veredicto: SMOTE AYUDÓ LEVEMENTE ✓
 MODELO GANADOR

 MODELO SELECCIONADO: Logistic Regression
   F1-Sco

# **17. FEATURE IMPORTANCE**

In [92]:
# FEATURE IMPORTANCE: LOGISTIC REGRESSION (GANADOR)

print(" FEATURE IMPORTANCE - Logistic Regression (MODELO GANADOR)")

# Extraer coeficientes
coeficientes_lr = pd.DataFrame({
    'Feature': X_train_balanced.columns,
    'Coeficiente': modelo_lr.coef_[0],
    'Abs_Coeficiente': np.abs(modelo_lr.coef_[0])
})

# Ordenar por importancia absoluta
coeficientes_lr = coeficientes_lr.sort_values('Abs_Coeficiente', ascending=False)

print("\n TOP 10 FEATURES MÁS IMPORTANTES:")
print(coeficientes_lr.head(10).to_string(index=False))

# INTERPRETACIÓN DE COEFICIENTES


print(" INTERPRETACIÓN DE COEFICIENTES")

print("\n LEYENDA:")
print("   - Coeficiente POSITIVO (+): Aumenta el riesgo de churn")
print("   - Coeficiente NEGATIVO (-): Reduce el riesgo de churn")
print("   - Valor absoluto más alto: Mayor influencia en la predicción")

# Separar en positivos y negativos
features_riesgo = coeficientes_lr[coeficientes_lr['Coeficiente'] > 0].head(5)
features_proteccion = coeficientes_lr[coeficientes_lr['Coeficiente'] < 0].head(5)

print(" TOP 5 FACTORES DE RIESGO (aumentan probabilidad de churn)")

for idx, row in features_riesgo.iterrows():
    feature = row['Feature']
    coef = row['Coeficiente']

    # Interpretación por feature
    if feature == 'tipo_contrato_mensual':
        explicacion = "Clientes con contrato mensual son más propensos a cancelar (menor compromiso)"
    elif feature == 'facturas_impagas':
        explicacion = "Más facturas impagas → Mayor riesgo de churn (problema de pago)"
    elif feature == 'cambios_plan':
        explicacion = "Clientes que cambian mucho de plan → Insatisfacción constante"
    elif feature == 'cliente_problematico':
        explicacion = "Clientes con muchos tickets + poco uso → Alto riesgo de abandonar"
    elif feature == 'plan_estandar':
        explicacion = "Plan estándar tiene más churn que básico (expectativas no cumplidas)"
    elif feature == 'premium_mensual':
        explicacion = "Premium con contrato mensual → Combinación de riesgo"
    elif feature == 'early_churn_risk':
        explicacion = "Cliente nuevo con problemas de pago → Alto riesgo temprano"
    else:
        explicacion = "Factor de riesgo identificado"

    print(f"\n{feature}")
    print(f"   Coeficiente: {coef:+.4f}")
    print(f"   Impacto: {'⬆️'*min(int(abs(coef)*5)+1, 5)}")
    print(f"   → {explicacion}")

print(" TOP 5 FACTORES DE PROTECCIÓN (reducen probabilidad de churn)")

for idx, row in features_proteccion.iterrows():
    feature = row['Feature']
    coef = row['Coeficiente']

    # Interpretación por feature
    if feature == 'canal_adquisicion_referido':
        explicacion = "Clientes referidos tienen mayor lealtad (confianza previa)"
    elif 'metodo_pago_tarjeta' in feature:
        explicacion = "Pagos con tarjeta → Mayor compromiso y facilidad de pago"
    elif feature == 'metodo_pago_transferencia_bancaria':
        explicacion = "Transferencia bancaria → Cliente más estable"
    elif feature == 'plan_premium':
        explicacion = "Plan premium → Cliente de mayor valor y compromiso"
    elif feature == 'frecuencia_uso':
        explicacion = "Mayor uso del servicio → Menor riesgo de cancelar"
    elif feature == 'antiguedad':
        explicacion = "Clientes más antiguos son más leales"
    else:
        explicacion = "Factor de protección identificado"

    print(f"\n{feature}")
    print(f"   Coeficiente: {coef:+.4f}")
    print(f"   Impacto: {'⬇️'*min(int(abs(coef)*5)+1, 5)}")
    print(f"   → {explicacion}")

# FEATURE IMPORTANCE: RANDOM FOREST (COMPARACIÓN)

print(" FEATURE IMPORTANCE - Random Forest (COMPARACIÓN)")

importancia_rf = pd.DataFrame({
    'Feature': X_train_balanced.columns,
    'Importancia': modelo_rf.feature_importances_
}).sort_values('Importancia', ascending=False)

print("\n TOP 10 FEATURES MÁS IMPORTANTES (por Gini):")
print(importancia_rf.head(10).to_string(index=False))

# COMPARACIÓN: LR vs RF

print(" COMPARACIÓN: Logistic Regression vs Random Forest")

# TOP 5 de cada modelo
top5_lr = coeficientes_lr.head(5)['Feature'].tolist()
top5_rf = importancia_rf.head(5)['Feature'].tolist()

print("\n TOP 5 según Logistic Regression:")
for i, feat in enumerate(top5_lr, 1):
    print(f"   {i}. {feat}")

print("\n TOP 5 según Random Forest:")
for i, feat in enumerate(top5_rf, 1):
    print(f"   {i}. {feat}")

# Features en común
comunes = set(top5_lr) & set(top5_rf)
print(f"\n Features en ambos TOP 5: {len(comunes)}")
if comunes:
    for feat in comunes:
        print(f"   - {feat}")

# RESUMEN EJECUTIVO DE FEATURES

print(" RESUMEN EJECUTIVO: Features más Importantes")

print("\n VARIABLES CLAVE IDENTIFICADAS:")

print("\n1️ TIPO DE CONTRATO")
print("   → Contratos mensuales tienen 2-3x más riesgo que anuales")
print("   → Recomendación: Incentivar contratos anuales con descuentos")

print("\n2️ FACTURAS IMPAGAS")
print("   → Cada factura impaga aumenta significativamente el riesgo")
print("   → Recomendación: Sistema de alertas tempranas por morosidad")

print("\n3️ CANAL DE ADQUISICIÓN")
print("   → Clientes referidos son los más leales")
print("   → Recomendación: Programa de referidos robusto")

print("\n4️ MÉTODO DE PAGO")
print("   → Tarjetas (crédito/débito) reducen churn")
print("   → Recomendación: Incentivar pago automático con tarjeta")

print("\n5️ FRICCIÓN DEL SERVICIO")
print("   → Alta proporción tickets/uso indica insatisfacción")
print("   → Recomendación: Mejora en calidad de servicio y soporte")

# FEATURES ENGINEERIZADAS: IMPACTO

print(" IMPACTO DE FEATURES ENGINEERIZADAS")

features_eng = ['friccion_del_servicio', 'ratio_valor_uso', 'cliente_problematico',
                'early_churn_risk', 'premium_mensual']

print("\n Ranking de Features Engineerizadas:")
for feat in features_eng:
    if feat in coeficientes_lr['Feature'].values:
        coef = coeficientes_lr[coeficientes_lr['Feature'] == feat]['Coeficiente'].values[0]
        abs_coef = abs(coef)
        rank = coeficientes_lr[coeficientes_lr['Feature'] == feat].index[0] + 1

        print(f"\n{feat}:")
        print(f"   Ranking: #{rank} de {len(coeficientes_lr)}")
        print(f"   Coeficiente: {coef:+.4f}")
        print(f"   Impacto: {'Alto' if abs_coef > 0.4 else 'Moderado' if abs_coef > 0.2 else 'Bajo'}")



 FEATURE IMPORTANCE - Logistic Regression (MODELO GANADOR)

 TOP 10 FEATURES MÁS IMPORTANTES:
                         Feature  Coeficiente  Abs_Coeficiente
           tipo_contrato_mensual     1.092044         1.092044
      canal_adquisicion_referido    -0.806105         0.806105
     metodo_pago_tarjeta_credito    -0.613347         0.613347
                    plan_premium    -0.571252         0.571252
                facturas_impagas     0.544999         0.544999
      metodo_pago_tarjeta_debito    -0.487053         0.487053
            cliente_problematico     0.337540         0.337540
canal_adquisicion_redes_sociales    -0.328355         0.328355
           canal_adquisicion_web    -0.251799         0.251799
                    cambios_plan     0.182789         0.182789
 INTERPRETACIÓN DE COEFICIENTES

 LEYENDA:
   - Coeficiente POSITIVO (+): Aumenta el riesgo de churn
   - Coeficiente NEGATIVO (-): Reduce el riesgo de churn
   - Valor absoluto más alto: Mayor influencia en la pr